In [2]:
import os
import sys
import numpy as np
import pandas as pd
import random

from argparse import ArgumentParser
import datetime

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.backends.cudnn as cudnn

from pytvision.transforms import transforms as mtrans
from pytvision import visualization as view

sys.path.append('../')
from torchlib.datasets.datasets import ATLASDataset 
from torchlib.neuralnet import NeuralNetClassifier
from misc import get_transforms_det

from sklearn import metrics

In [3]:
pathname = os.path.expanduser( '~/.datasets' )
projectnet='../out/netruns'
no_cuda=False
seed=1
gpu=1
batch_size=60
workers=10
num_input_channels=3

print(pathname)
print(projectnet)

/root/.datasets
../out/netruns


In [5]:
!ls ../out/netruns/

atlas_baseline_preactresnet18_dice_adam_atlas_001
atlas_baseline_preactresnet18_dice_sgd_atlas_001
atlas_baseline_preactresnet18_focal_adam_atlas_001
atlas_baseline_preactresnet18_focal_sgd_atlas_001
atlas_baseline_preactresnet18_l1_sgd_atlas_001
atlas_baseline_preactresnet18_mse_sgd_atlas_001
baseline_preactresnet18_mse_sgd_atlas_001


In [6]:
name = 'atlas_baseline_preactresnet18_focal_adam_atlas_001'
project = os.path.join(projectnet, name)
model = 'model_best.pth.tar' #'chk000025.pth.tar', model_best
pathmodels = os.path.join(project,'models',model)

print(project)
print(pathmodels)

../out/netruns/atlas_baseline_preactresnet18_focal_adam_atlas_001
../out/netruns/atlas_baseline_preactresnet18_focal_adam_atlas_001/models/model_best.pth.tar


In [7]:
# neuralnet
network = NeuralNetClassifier(
    patchproject=project,
    nameproject=name,
    no_cuda=no_cuda,
    seed=seed,
    gpu=gpu
    )

cudnn.benchmark = True

# load model
if network.load( pathmodels ) is not True:
    assert(False)

Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


=> loading checkpoint '../out/netruns/atlas_baseline_preactresnet18_focal_adam_atlas_001/models/model_best.pth.tar'
=> loaded checkpoint for preactresnet18 arch!


In [41]:
pathdata='~/.kaggle/competitions/human-protein-atlas-image-classification'
name_dataset='atlas'
subset='test'

data = ATLASDataset(        
    path=pathdata, 
    train=False,
    folders_images='test', 
    metadata='sample_submission.csv',
    #count=100,
    num_channels=3,
    transform=get_transforms_det( network.size_input ), #get_transforms_aug
    )

dataloader = DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=workers )

print('Data loader ')
print(len(dataloader))
print(len(data))

Data loader 
196
11702


In [42]:
Id, Yhat, Y = network.predict( dataloader )
df = pd.DataFrame( np.concatenate((Id[:,np.newaxis], Yhat), axis=1) )
df.to_csv( os.path.join(project , '{}_{}_{}_dp.csv'.format(subset,name,name_dataset)), index=False, encoding='utf-8')        
print('dir: {}'.format(project))
print('DONE!!!')

100%|██████████| 196/196 [01:24<00:00,  2.31it/s]


dir: ../out/netruns/atlas_baseline_preactresnet18_focal_adam_atlas_001
DONE!!!


In [43]:
df = pd.read_csv( os.path.join(project , '{}_{}_{}_dp.csv'.format(subset, name, name_dataset))  )
df.head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0.281559,0.130533,0.633081,0.542131,0.184754,0.142710,0.057011,0.051847,0.006822,...,0.016103,0.062693,0.009073,0.081635,0.058379,0.119023,0.025292,0.140276,0.029296,0.002908
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0.311054,0.268357,0.344294,0.411465,0.331445,0.466924,0.229786,0.158757,0.044935,...,0.219351,0.259326,0.054188,0.205345,0.144533,0.158114,0.134517,0.469081,0.145158,0.020674
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0.697853,0.222277,0.286408,0.146643,0.150515,0.230809,0.234871,0.256254,0.015285,...,0.153982,0.206302,0.042351,0.330271,0.149011,0.231462,0.040409,0.703671,0.061319,0.012203
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0.547530,0.267578,0.278272,0.127871,0.174223,0.233563,0.181049,0.235094,0.023304,...,0.185311,0.169046,0.066319,0.310326,0.112472,0.292982,0.056462,0.418399,0.157046,0.038610
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0.539889,0.168793,0.198296,0.126466,0.115165,0.192996,0.169623,0.248586,0.044657,...,0.152686,0.138650,0.124900,0.374852,0.146164,0.418872,0.063119,0.406983,0.119180,0.037956


In [45]:
result = df.as_matrix()
iDs = result[:,0]
Yhat = result[:,1:]

print(iD.shape)
print(Yhat.shape)


(100,)
(11702, 28)


In [47]:
(Yhat>0.4).sum(axis=0)

array([6966,  486, 1041,  410,  361,  697,  352,  365,    0,    0,    0,
        309,  144,   35,  352,    0,    0,   53,   53,   68,    2, 2081,
         48, 1441,   90, 7429,    0,    0])

In [84]:
pred = [ ' '.join( map(str,np.where( yhat > 0.4 )[0]) ) for yhat in  Yhat]
#code  = ' '.join( map(str, code) )
len(np.where( np.array([1,2,3]) < 0 )[0]) 

0

In [91]:
submission_filepath = 'submission.csv'
th=0.4
#submission = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0])  if len(np.where( yhat > th )[0])>0 else '0'  ) }  for (Id, yhat) in zip( iDs,Yhat) ]
submission = [ {'Id': Id, 'Predicted': ' '.join( map(str, np.where( yhat > th )[0]) ) }  for (Id, yhat) in zip( iDs,Yhat) ]
submission = pd.DataFrame( submission )
submission.to_csv(submission_filepath, index=False, encoding='utf-8')
print('SAVE!!!')
submission


SAVE!!!


,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2 3
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,3 5 25
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0 23 25
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,4 25
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0 25
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0 25
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,25


In [92]:
!kaggle competitions submit -c human-protein-atlas-image-classification -f submission.csv -m "first-submission-test"

100%|#########################################| 480k/480k [00:03<00:00, 134kB/s]
Successfully submitted to Human Protein Atlas Image Classification

In [59]:
submission = pd.read_csv('/root/.kaggle/competitions/human-protein-atlas-image-classification/sample_submission.csv')
submission

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,0
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,0
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,0
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,0
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0
